## Data Preparation 

In [ ]:
#Load packages
#Data handling
import requests
import numpy as np
import pandas as pd
import string as st
import os
import csv
import re
import random

from sklearn import datasets
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt 

# for naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [2]:
#Set path
data = '/Users/caitdreisbach/Dropbox (Personal)/Academic/University of Virginia/Data Science Institute/Spring 2018/SYS 6016/data.csv'
labels = '/Users/caitdreisbach/Dropbox (Personal)/Academic/University of Virginia/Data Science Institute/Spring 2018/SYS 6016/labels.csv'

In [3]:
df1 = pd.read_csv(data, encoding= "ISO-8859-1", low_memory=False)

In [4]:
labels = pd.read_csv(labels, encoding= "ISO-8859-1", low_memory=False)

In [5]:
df1.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [6]:
labels.head()

,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA


In [7]:
labels.shape #801 samples

(801, 2)

In [8]:
df1["Class"] = labels["Class"]

In [9]:
df2 = df1.drop("Unnamed: 0", axis=1)

In [10]:
df2.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530,Class
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0,PRAD
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0,LUAD
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0,PRAD
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0,PRAD
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0,BRCA


In [11]:
df2["Class"].dtypes

dtype('O')

In [12]:
df2["Class"] = df2["Class"].astype('category')

In [13]:
df2["Class"].dtypes

CategoricalDtype(categories=['BRCA', 'COAD', 'KIRC', 'LUAD', 'PRAD'], ordered=False)

In [14]:
df2["gene_2"].dtypes

dtype('float64')

In [15]:
df2.isnull().sum().sum() #no missing values

0

In [16]:
x = df2.loc[:,:"gene_20530"]

In [17]:
x.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [18]:
y = df2.loc[:,"Class"]

In [19]:
y.head()

0    PRAD
1    LUAD
2    PRAD
3    PRAD
4    BRCA
Name: Class, dtype: category
Categories (5, object): [BRCA, COAD, KIRC, LUAD, PRAD]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [21]:
x_train.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
159,0.0,3.755902,4.589122,6.440992,9.438946,0.0,8.335864,0.560128,0.0,0.0,...,5.667046,8.342204,10.147587,6.502369,9.999013,12.114276,10.344673,9.110374,5.013878,0.0
337,0.0,2.683337,2.982327,7.381655,10.126149,0.0,7.664561,0.887603,0.0,0.0,...,6.338184,9.942045,10.217049,5.470976,9.274786,11.685511,11.121378,9.343414,4.203702,0.0
77,0.0,0.000000,3.193236,5.329328,10.415277,0.0,9.464960,1.602504,0.0,0.0,...,4.484454,7.517913,10.505970,6.003074,9.110966,11.380445,8.939041,8.730046,3.724650,0.0
651,0.0,3.391204,3.313725,7.066369,9.295190,0.0,8.349126,0.535655,0.0,0.0,...,6.379971,9.071114,10.190158,5.493926,9.874213,12.310908,10.311499,9.783647,8.501881,0.0
283,0.0,3.188433,3.802876,5.223307,10.039467,0.0,8.277924,0.712464,0.0,0.0,...,5.441779,8.521754,10.124044,6.462131,9.511608,11.523645,9.764497,8.457689,6.508402,0.0


In [22]:
y_train.head()

159    LUAD
337    BRCA
77     KIRC
651    BRCA
283    LUAD
Name: Class, dtype: category
Categories (5, object): [BRCA, COAD, KIRC, LUAD, PRAD]

## Bayes Classifier

In [23]:
# Scale the data to be between -1 and 1
scaler = StandardScaler()
X=scaler.fit_transform(x)
X

array([[-0.19479935, -0.82802988,  0.15980044, ..., -1.18793812,
        -0.11648251, -0.26190144],
       [-0.19479935, -2.01501735, -1.415042  , ..., -0.34227662,
        -1.65688871, -0.26190144],
       [-0.19479935,  0.41734754,  1.15673547, ...,  0.88686027,
        -1.85526414, -0.26190144],
       ..., 
       [-0.19479935,  0.19888076,  0.57481583, ..., -0.22008186,
        -0.41046699,  1.3485582 ],
       [-0.19479935, -0.35045311, -0.28863152, ...,  1.43719268,
         0.09195083, -0.26190144],
       [-0.19479935, -0.57135218,  0.66725377, ...,  0.45087581,
        -0.47161901, -0.26190144]])

In [24]:
# split the training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.2,random_state=4)

### Gaussian Naive Bayes

In [25]:
# chose the naive bayes model for comparison
model_naive = GaussianNB()
model_naive.fit(X_train, Y_train)

GaussianNB(priors=None)

In [26]:
# compute the positive class predicition probabilities 
y_prob = model_naive.predict_proba(X_test)[:,1] 
y_pred = np.where(y_prob > 0.5, 1, 0) 
model_naive.score(X_test, y_pred)

/Users/caitdreisbach/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:181: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [27]:
# compute accuracy scores with cross validation
scores = cross_val_score(model_naive, X, y, cv=10, scoring='accuracy')
print(scores)

[ 0.74390244  0.71604938  0.61728395  0.61728395  0.61728395  0.69135802
  0.6835443   0.65822785  0.73076923  0.66666667]


In [28]:
# find the mean of the scores
scores.mean()

0.6742369747618282

### Bernoulli Naive Bayes 

In [35]:
model_naive_b = BernoulliNB()
model_naive_b.fit(X_train, Y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [37]:
# compute the positive class predicition probabilities 
y_prob = model_naive_b.predict_proba(X_test)[:,1] 
y_pred = np.where(y_prob > 0.5, 1, 0) 
model_naive.score(X_test, y_pred)

/Users/caitdreisbach/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:181: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [39]:
# compute accuracy scores with cross validation
scores_b = cross_val_score(model_naive_b, X, y, cv=10, scoring='accuracy')
print(scores_b)

[ 0.97560976  0.98765432  0.98765432  0.98765432  0.97530864  1.          1.
  0.97468354  0.98717949  0.94871795]


In [40]:
# find the mean of the scores
scores_b.mean()

0.98244623412370657